0.4.1


In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
# Import comet_ml in the top of your file
from comet_ml import Experiment
##Needs to be imported before sklearn
from sklearn.model_selection import train_test_split
import sklearn

import torch
import torch.utils.data
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch import nn, optim, sigmoid
from torch.nn import functional as F
from torchvision import datasets, transforms
from torch.nn import modules

from torch.autograd import Variable

#from torchaudio import transforms
#from torchaudio import Datasets

import os
import sys
import shutil
from glob import glob
import datetime
import re
from tqdm import tqdm
import time


#from torchviz import make_dot, make_dot_from_trace


In [2]:
# Check device type


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
device


device(type='cpu')

In [4]:
def data_creator(T = 20, L = 10000, N = 1000):
    np.random.seed(2)



    x = np.empty((N, L), 'int64')
    x[:] = np.array(range(L)) + np.random.randint(-4 * T, 4 * T, N).reshape(N, 1)
    data = np.sin(x / 1.0 / T).astype('float64')
    data = sklearn.preprocessing.normalize(data)

    torch.save(data, open('traindata.pt', 'wb'))

In [5]:
# input to LSTM is sequence length, batch, input_size   
#sequence length ---- how many time steps RNN
#batch 


In [6]:
class AE_model(nn.Module):
    def __init__(self):
        super(AE_model, self).__init__()

        self.fc1 = nn.Linear(10000, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 10000)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if torch.cuda.is_available():
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar



In [ ]:
class RVAE(nn.Module):
    nn.Sequential(
    
    )

In [22]:
class sineDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.data = torch.load('traindata.pt')
        self.label = []
        vals = len(self.data)
        for val in range(vals):
            self.label.append(val)

        self.data, self.label = map(torch.FloatTensor, (self.data, self.label))
        
        

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]
        

In [23]:

def get_model():
    
    model = AE_model().to(device)

    
    return model, optim.Adam(model.parameters(), lr = lr)


In [24]:
def get_data(train_dataset, test_dataset, bs):
    return (DataLoader(train_dataset, batch_size = batch_size),
    DataLoader(test_dataset, batch_size = batch_size * 2))

In [25]:
###Need data_creator function

data_creator()

In [26]:
hidden_dim = 10000
latent = 5
input_dim = 5
epochs = 10000
seq_len = 10
batch_size = 32
input_size = 997

features = 997
latent =50


lr = 1e-6

In [27]:
 
full_dataset = sineDataset( )
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])
train_dl, test_dl = get_data(train_dataset, test_dataset, batch_size)


In [28]:
def loss_function(recon_batch, mu, logvar, x):
    """
    recon_x: generating images
    x: origin images
    mu: latent mean
    logvar: latent log variance
    """
    MSE = F.binary_cross_entropy(recon_batch, x)  # mse loss
    # loss = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)
    # KL divergence
    return MSE + KLD

In [29]:
def loss_batch(model, loss_function, xb, opt=None):
    recon_batch, mu,logvar = model(xb)
    loss = loss_function(recon_batch, mu,logvar, xb)
   
    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()
        
    return loss.item(), len(xb)

In [30]:
def fit(epochs, model, loss_func, opt, train_dl, test_dl):
    for epoch in range(epochs):
        model.train()
        
        for xb in train_dl: 
            
            loss_batch(model, loss_func, xb, opt)
            model.eval()
            with torch.no_grad():
                losses,nums = zip(*[loss_batch(model, loss_func, xb)
                                    for xb in test_dl])
            val_loss = np.sum(np.multiply(losses,nums)) / np.sum(nums)
            print(epoch, val_loss)

## Full training in 3 lines of code

In [31]:

model, opt = get_model()
#x = torch.randn(1, 10000)
#make_dot(model(x), params=dict(model.named_parameters()))

       
    

In [32]:
fit(epochs, model, loss_function, opt, train_dl, test_dl)

0 1.4663240838050842
0 1.464477379322052
0 1.463164553642273
0 1.4615251898765564
0 1.4604030585289
0 1.458550145626068
0 1.4568650484085084
0 1.4558627533912658
0 1.454238338470459
0 1.452788963317871
0 1.4512803220748902
0 1.449676902294159
0 1.4485812425613402
0 1.4470227694511413
0 1.4456518840789796
0 1.444277195930481
0 1.442923254966736
0 1.4415382027626038


KeyboardInterrupt: 


# Page Break



























































In [ ]:

class AE(nn.Module):
    def __init__(self, input_size, hidden_size, latent):
        super(AE, self).__init__()
        self.hidden_size = hidden_size
        self.latent = latent
        self.gru = nn.GRU(input_size, self.hidden_size)
        self.linear = nn.Linear(input_size, self.latent)
        self.gru2 = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, output_size)
        
        
    def forward(self, input):
        
        output, hidden = self.gru(input, self.hidden_size)
        fc1 = self.linear(output)
        output2, hidden = self.gru(fc1, self.hidden_size)
        output = self.softmax(self.out(output[0]))
        
        return output, hidden
    def initHidden(self):
        return torch.zeros(1,1, self.hidden_size, device = device)

In [ ]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x, size_average=False)

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    #KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE #+ KLD

# Page Break


In [ ]:
# Get x values of the sine wave

time        = np.arange(0, 1000, 0.1);

# Amplitude of the sine wave is sine of a variable like time

amplitude   = np.sin(time)
#Create multiple amplitude arrays of varying size
amplitude_2 = amplitude * 2
amplitude_4 = amplitude *4
amplitude_03 = amplitude * 1/3
amplitude_02 = amplitude * 1/2
amplitude_8 = amplitude * 8
amplitudes = {'amplitude_2':amplitude_2, 
              'amplitude_4':amplitude_4, 
              'amplitude_8':amplitude_8, 
              'amplitude_02':amplitude_02,
              'amplitude_03':amplitude_03}

amplitudes_nd = [amplitude_2, 
              amplitude_4, 
              amplitude_8, 
              amplitude_02,
              amplitude_03]


eps = 2

In [6]:
df = pd.DataFrame(amplitudes)

In [8]:
df

,amplitude_2,amplitude_4,amplitude_8,amplitude_02,amplitude_03
0,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.199667,0.399334,0.798667,0.049917,0.033278
2,0.397339,0.794677,1.589355,0.099335,0.066223
3,0.591040,1.182081,2.364162,0.147760,0.098507
4,0.778837,1.557673,3.115347,0.194709,0.129806
5,0.958851,1.917702,3.835404,0.239713,0.159809
6,1.129285,2.258570,4.517140,0.282321,0.188214
7,1.288435,2.576871,5.153741,0.322109,0.214739
8,1.434712,2.869424,5.738849,0.358678,0.239119
9,1.566654,3.133308,6.266615,0.391663,0.261109


In [12]:
torch_tensor = torch.tensor(df.values)

TypeError: 'module' object is not callable

In [ ]:
torch_tensor.type

In [ ]:
torch_tensor = torch_tensor.transpose(0, 1)

In [ ]:
#Hyper_Params
num_epochs = 5
learning_rate = 1e-6
log_interval = 100

In [ ]:

train = TensorsDataset(torch_tensor.float())

train_loader= torch.utils.data.DataLoader(train, batch_size=1)


In [ ]:
# get some random training images
dataiter = iter(train_loader)

In [ ]:

vals = dataiter.next()
print(vals)

In [ ]:

class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.rnn1 = nn.LSTM(10000, 400, num_layers = 2)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(20, 20)
        self.rnn3 = nn.LSTM(20, 400, num_layers = 1)
        self.rnn4 = nn.LSTM(400, 10000, num_layers = 2)

    def encode(self, x):
        h1 = F.relu(self.rnn1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        if self.training:
            std = torch.exp(0.5*logvar)
            eps = torch.randn_like(std)
            return eps.mul(std).add_(mu)
        else:
            return mu

    def decode(self, z):
        h3 = F.relu(self.rnn3(z))
        return F.sigmoid(self.rnn4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


model = VAE()
optimizer = optim.Adam(model.parameters(), lr=1e-3)


In [ ]:
seq_len = 1000
input_size = 1
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(train_loader):

        
        print(data.shape)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
epoch, train_loss / len(train_loader.dataset)))

In [ ]:


def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            data = data.view(-1,1,1)
            
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(batch_size, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [135]:
optimizer.zero_grad()
for epoch in range(1000):
    train(epoch)
    #test(epoch)


torch.Size([1, 10000])


RuntimeError: input must have 3 dimensions, got 2

In [22]:
# Get x values of the sine wave

time        = np.arange(0, 1000, 0.1);

# Amplitude of the sine wave is sine of a variable like time

x1   = np.sin(time)
#Create multiple amplitude arrays of varying size
x2 = x1 * 2
x3 = x1 *4
y = x1 * 1/3



eps = 2

In [23]:


class DataGenerator():
    def __init__(self, dset, bs=1):
        self.dset = torch.LongTensor(dset).cuda()
        self.len = len(self.dset)
        self.idx = 0
        self.bs = bs
    def __len__(self):
        return len(self.dset)
    def __iter__(self):
        return self
    def __next__(self):
        batch = [t for t in [torch.LongTensor(s.T).cuda() for s in np.stack([self.dset[i] for i in range(self.idx, self.idx + self.bs)]).T]] 
        self.idx = self.idx + self.bs
        if self.idx > self.len - self.bs:
            raise StopIteration
        return batch



In [27]:
it = DataGenerator(np.stack([x1,x2,x3,y], axis=1), bs=2)

In [86]:

*Xs, yt = next(it)
print(Xs)
#(*[Variable(x) for x in Xs])

[tensor([0, 0], device='cuda:0'), tensor([-1, -1], device='cuda:0'), tensor([-3, -3], device='cuda:0')]
